In [ ]:
########new dataset
#images
import argparse
import pickle
from torchvision.models import detection
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
from stacked_hourglass.datasets import generic
import argparse
import glob
from albumentations.core.composition import KeypointParams
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import numpy as np
from stacked_hourglass.predictor import HumanPosePredictor, GenericPosePredictor
from stacked_hourglass.model import hg2
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import csv
import glob
import random
import copy
from PIL import Image
import albumentations as A
from stacked_hourglass.utils import finetune
from stacked_hourglass import train as t
from stacked_hourglass.datasets.mpii import Mpii
from torch.nn import DataParallel
from stacked_hourglass.utils.logger import Logger
import os
from stacked_hourglass.utils.misc import save_checkpoint, adjust_learning_rate
from stacked_hourglass.train import do_training_epoch, do_validation_epoch
import tqdm
import argparse
from stacked_hourglass.datasets.mpii import MPII_JOINT_NAMES
import cv2
from torch.optim.rmsprop import RMSprop
from torch.utils.tensorboard import SummaryWriter
import random

#####################################################################################################
parser = argparse.ArgumentParser(description='Train a stacked hourglass model.')
# Model structure
parser.add_argument('--arch', '-a', metavar='ARCH', default='hg2',
                    choices=['hg1', 'hg2', 'hg8'],
                    help='model architecture')
# Training strategy
parser.add_argument('--input_shape', default=(256, 256), type=int, nargs='+',
                    help='Input shape of the model. Given as: (H, W)')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=20, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--train-batch', default=16, type=int, metavar='N',
                    help='train batchsize')
parser.add_argument('--test-batch', default=16, type=int, metavar='N',
                    help='test batchsize')
parser.add_argument('--lr', '--learning-rate', default=2.5e-4, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--weight-decay', '--wd', default=0, type=float,
                    metavar='W', help='weight decay (default: 0)')
parser.add_argument('--schedule', type=int, nargs='+', default=[200, 201],
                    help='Decrease learning rate at these epochs.')
parser.add_argument('--gamma', type=float, default=0.1,
                    help='LR is multiplied by gamma on schedule.')
# Miscs
parser.add_argument('-c', '--checkpoint', default='checkpoint/', type=str, metavar='PATH',
                    help='path to save checkpoint (default: checkpoint)')
parser.add_argument('--snapshot', default=0, type=int,
                    help='save models for every #snapshot epochs (default: 0)')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')


#heatmap check
# haetmap output of the pre-trained model
#from stacked_hourglass.train import generate_gaussian
from stacked_hourglass.predictor import HumanPosePredictor, GenericPosePredictor
device='cuda'
model1 = hg2(pretrained=True)
model1 = DataParallel(model1).to(device)
input = torch.empty((1,3,256,256), device=device, dtype=torch.float32)
#target = torch.zeros(1,20, 64, 64)
input[0],target,meta=next(iter(train_loader))[1]
#plt.imshow(np.array(input[0].to('cpu').permute(1,2,0)))
with torch.enable_grad():
        # Forward pass and loss calculation.
        output1 = model1(input)
'''for i in range(20):
    plt.subplot(4,5,i+1)
    plt.imshow(output1[0][0,i,:,:].to('cpu').detach().numpy())
plt.show()'''

# haetmap output of the trained model
#input = torch.empty((1,3,256,256), device=device, dtype=torch.float32)
#input[0]=predictor.prepare_image(torch.as_tensor(frame1))

#plt.imshow(np.array(input[0].to('cpu').permute(1,2,0)))

with torch.enable_grad():
        # Forward pass and loss calculation.
        output = model(input)
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.imshow(output[0][0,i,:,:].to('cpu').detach().numpy())
    #plt.imshow(target[i].to('cpu').detach().numpy())
plt.show()

#landmark check
image=cv2.imread('C:/Users/mobinamb/Desktop/Mobina work/pytorch pose estimation/transferlearning/pytorch-stacked-hourglass -571t/src/dataset/images/RGB1/s04_e02/colorImg716.jpg')
#image=cv2.resize(image,(256,256),interpolation = cv2.INTER_AREA)
device='cuda'
frame1 = np.moveaxis(image, -1, 0)
frame2 = torch.as_tensor(np.array(frame1, copy=True))

model = DataParallel(model).to(device)
predictor = HumanPosePredictor(model, device=device)
joints = predictor.estimate_joints(frame2, flip=False)
#image=np.array(input[0].to('cpu').permute(1,2,0))
for i in range(len(joints)):
    #plt.plot(int(joints[i][0]),int(joints[i][1]), color="white", linewidth=5)
    #plt.imshow(image)
    frame=cv2.circle(image,(int(joints[i][0]),int(joints[i][1])),radius=5,color=(255,0,255),thickness=-1) # predicted purple pretrained
plt.imshow(frame)